In [1]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
from nltk.tokenize import word_tokenize


In [2]:
train = pd.read_csv('train.tsv', sep="\t")
test = pd.read_csv('test.tsv', sep="\t")

train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [3]:
sub = pd.read_csv('sampleSubmission.csv')


In [4]:
test["Sentiment"] = -10
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,-10
1,156062,8545,An intermittently pleasing but mostly routine ...,-10
2,156063,8545,An,-10
3,156064,8545,intermittently pleasing but mostly routine effort,-10
4,156065,8545,intermittently pleasing but mostly routine,-10


In [5]:
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english')) 
not_delete = ["but", "shan't", "wasn't", "couldn't", "didn't", "hadn't", "against", "no", "haven't", "shouldn't", "needn't", "wouldn't", "aren't", "mightn't", "won't", "isn't", "hasn't", "don't", "mustn't", "doesn't", "not"]
stop_words = [w for w in stop_words if w not in not_delete]
print(stop_words)

df=pd.concat([train, test], ignore_index = True)

df = df.applymap(lambda s:s.lower() if type(s) == str else s)
#del test, train

['those', 'my', 'when', 'after', 'only', 'down', 'is', 'to', "weren't", "you're", 'of', 'own', 'as', 'out', 'themselves', 'has', 'what', 'his', 'y', 'himself', 'while', "you'll", 'doesn', "you've", 'herself', 'i', 'm', 'him', 'didn', 'all', 'over', 'until', 'few', 'such', 'itself', 'having', 'both', 'have', 'her', 'she', 'in', 'or', 'that', 'mustn', 'same', 'haven', 'mightn', "it's", 'they', 'your', 'yourselves', 'shan', 'now', 'do', 'by', 'should', 'he', 'll', 'with', 'me', 'during', 'o', 'any', 'each', "she's", 'nor', 'which', 'further', 'myself', 'them', 'above', 'where', 'this', 'don', 'ma', 'doing', 'weren', 'some', 'needn', 't', 'can', 'it', 'into', 'hers', 'up', 'through', 'did', 'why', 'more', 're', 'wasn', 'there', 'we', 'had', 'before', 'been', 'yours', 'whom', "you'd", "should've", 'aren', 'does', 'so', 'below', 'theirs', 'hadn', 'ours', 'these', 've', 'than', 'again', 'was', 'under', 'shouldn', 'most', 'the', 'and', 'will', 'our', 'who', 'ourselves', 'won', 'were', 'on', 'a

In [6]:
porter = PorterStemmer()
def stem_sentence(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        if word not in stop_words:
            stem_sentence.append(porter.stem(word))
            stem_sentence.append(" ")
    return "".join(stem_sentence)

In [7]:
def clean_review(phrase):
    clean_review = [ ]
    for i in range(0, len(phrase)):
        review = str(phrase[i])
        review = re.sub("[^a-zA-Z]", " ", review)
        stem_review = stem_sentence(review)
        clean_review.append(stem_review)
    return clean_review

In [8]:
df['clean_review'] = clean_review(df.Phrase.values)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment,clean_review
0,1,1,a series of escapades demonstrating the adage ...,1,seri escapad demonstr adag good goos also good...
1,2,1,a series of escapades demonstrating the adage ...,2,seri escapad demonstr adag good goos
2,3,1,a series,2,seri
3,4,1,a,2,
4,5,1,series,2,seri


In [9]:
df.shape

(222352, 5)

In [10]:
test = df[df.Sentiment == -10]
test.drop("Sentiment", axis =1, inplace = True)
print(test.shape)
test.head()

(66292, 4)


/Users/Junda/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PhraseId,SentenceId,Phrase,clean_review
156060,156061,8545,an intermittently pleasing but mostly routine ...,intermitt pleas but mostli routin effort
156061,156062,8545,an intermittently pleasing but mostly routine ...,intermitt pleas but mostli routin effort
156062,156063,8545,an,
156063,156064,8545,intermittently pleasing but mostly routine effort,intermitt pleas but mostli routin effort
156064,156065,8545,intermittently pleasing but mostly routine,intermitt pleas but mostli routin


In [11]:
train = df[df.Sentiment != -10]
print(train.shape)



(156060, 5)


In [12]:
whole_text = list(df.clean_review.values)
sen_len = []
for text in whole_text:
    word = word_tokenize(text)
    sen_len.append(len(word))
max_len = np.max(sen_len)
max_len

30

In [13]:
from keras.utils import to_categorical
X_test = test.clean_review.values


Using TensorFlow backend.


In [14]:
from sklearn.model_selection import train_test_split
training_set, validation_set = train_test_split(train, test_size = 0.2)
X_train = training_set.clean_review.values
Y_train = to_categorical(training_set.Sentiment.values)
X_val = validation_set.clean_review.values
Y_val = to_categorical(validation_set.Sentiment.values)

In [15]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(124848,)
(124848, 5)
(31212,)
(31212, 5)


In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [17]:
tk = Tokenizer(lower=True, filters='')
tk.fit_on_texts(whole_text)
X_train = tk.texts_to_sequences(X_train)
X_val = tk.texts_to_sequences(X_val)
X_test = tk.texts_to_sequences(X_test)

In [18]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_val = pad_sequences(X_val, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
print(X_train.shape, X_val.shape, X_test.shape)

(124848, 30) (31212, 30) (66292, 30)


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras import backend

In [20]:
from keras.optimizers import Adam
model1 = Sequential()
model1.add(Embedding(15000, 128))
model1.add(LSTM(64, dropout = 0.4, recurrent_dropout=0.4, return_sequences=True))
model1.add(LSTM(32, dropout = 0.5, recurrent_dropout = 0.5, return_sequences=False))
model1.add(Dense(5, activation='softmax'))
opt = Adam(lr=0.001, decay=1e-5)
model1.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1920000   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 64)          49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 1,981,989
Trainable params: 1,981,989
Non-trainable params: 0
_________________________________________________________________


In [21]:
model1.fit(X_train, Y_train, batch_size = 32, epochs = 3, verbose = 1, validation_data=(X_val, Y_val))

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 273s 2ms/step - loss: 0.9906 - acc: 0.5988 - val_loss: 0.8703 - val_acc: 0.6449
Epoch 2/3
124848/124848 [==============================] - 285s 2ms/step - loss: 0.8357 - acc: 0.6558 - val_loss: 0.8391 - val_acc: 0.6535
Epoch 3/3
124848/124848 [==============================] - 270s 2ms/step - loss: 0.7823 - acc: 0.6787 - val_loss: 0.8194 - val_acc: 0.6624


In [22]:
p1 = model1.predict_classes(X_test, verbose = 1)


66292/66292 [==============================] - 19s 282us/step


In [23]:
sub.Sentiment = p1
sub.to_csv('sub1.csv', index = False)
sub.head()

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
